In [143]:
import numpy as np
import aplpy
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams, style
import astropy.io.fits as pyfits
from astropy import units as u
import pyregion
from astropy.wcs import WCS
from sys import argv
import time
import os
import warnings
from astropy import units as u
from astropy.coordinates import SkyCoord
import re

In [240]:
#---------------------------------------
# Input some parameters from parameter.ipynb
#---------------------------------------
%run parameter.ipynb

#---------------------------------------
# Input parameters used in this program:

###-----------
# Path name
# path_image, path_BT, path_figure, path_samping

###-----------
# File name
# outname_b4,  outname_b7
# BT_B4, BT_B7, BT_slt_image, BT_div_image
# Mask_image

###-----------
# criteria and rms for the imaging
# [Jy beam-1]
# rms_I_b4, rms_Q_b4, rms_PI_b4, I_clip_b4, PI_lowclip_b4, PI_highclip_b4,
# rms_I_b7, rms_Q_b7, rms_PI_b7, I_clip_b7, PI_lowclip_b7, PI_highclip_b7
# sampling all, sampling_b4, sampling_b7

###-----------
# some parameters about the targets
# dist [pc]
# ra_4A1, ra_4A2, ra_4A1_4A2
# dec_4A1, dec_4A2, dec_4A1_4A2 

In [241]:
#----------------------------------------
# define the functions
#----------------------------------------
def nyquist_sampling(outname, I_clip, PI_lowclip, PI_highclip, \
                     rms_I, rms_Q, rms_PI, sampling_interval, Select_region):
    
    ###------------------------------
    # Set the imagename
    I_image = '%s%s_I.image.fits'%(path_image, outname)
    Q_image = '%s%s_Q.image.fits'%(path_image, outname)
    U_image = '%s%s_U.image.fits'%(path_image, outname)
    PI_image = '%s%s_PI.image.fits'%(path_image, outname)
    PA_image = '%s%s_PA.image.fits'%(path_image, outname)
    Per_image = '%s%s_Per.image.fits'%(path_image, outname)
    BT_image = '%s%s_BT.fits'%(path_BT, outname)        
    
    ###------------------------------
    # Get the FITS and convert them from 4 to 2 dimensions (y, x)
    I_data = Getdata(I_image)
    Q_data = Getdata(Q_image)
    U_data = Getdata(U_image)
    PI_data = Getdata(PI_image)
    PA_data = Getdata(PA_image)
    Per_data = Getdata(Per_image)
    BT_data = Getdata(BT_image)
    BT_slt_data = Getdata(BT_slt_image)
    BT_div_data = Getdata(BT_div_image) 
    Mask_data = Getdata(Mask_image)
    Mask_4A1_data = Getdata(Mask_4A1_image)
    Mask_4A2_data = Getdata(Mask_4A2_image)  
    
    # Get the header
    I_hd = pyfits.getheader(I_image)

    # Fitsfile have 2 dimension: (y, x)
    Ny, Nx = I_data.shape
    nx, ny = np.meshgrid(np.arange(Nx), np.arange(Ny))
    
    # Get the coordinates (of world coor. system) of Stokes I intensity map
    wcs = WCS(I_hd)
    Coord = SkyCoord.from_pixel(nx, ny, wcs)
    RA_data = Coord.ra.deg
    Dec_data = Coord.dec.deg     
    
    ###------------------------------
    # Set the different criteria
    # standard criteria
    crit_stand = (I_data > I_clip) &\
                (PI_data > PI_lowclip) &\
                (PI_data < PI_highclip) &\
                (np.isnan(Per_data) == False) &\
                (nx % sampling_interval == 0) &\
                (ny % sampling_interval == 0)
    
    # Specific criterion: split the 4A1 & 4A2
    # select data within the specific B4 contour (4A1 & 4A2)
    crit_mask = Mask_data > 0    
    # select 4A1 only
    crit_mask_4A1 = Mask_4A1_data > 0
    # select 4A2 only
    crit_mask_4A2 = Mask_4A2_data > 0
    
    ###------------------------------
    # Criteria selection
    if Select_region == 'mask':
        crit = crit_stand & crit_mask
    elif Select_region == 'mask_4A1':
        crit = crit_stand & crit_mask_4A1
    elif Select_region == 'mask_4A2':
        crit = crit_stand & crit_mask_4A2
    else:
        crit = crit_stand
    
    ###------------------------------
    # Get the selected pixel value based on the criteria
    I_crit_data = I_data[np.where(crit)]               # Stokes I [Jy beam-1]
    Q_crit_data = Q_data[np.where(crit)]               # Stokes Q [Jy beam-1]
    U_crit_data = U_data[np.where(crit)]               # Stokes U [Jy beam-1]
    PI_crit_data = PI_data[np.where(crit)]             # Polarized intensity [Jy beam-1]
    PA_crit_data = PA_data[np.where(crit)]             # Polarization angle [deg]
    Per_crit_data = Per_data[np.where(crit)]           # Polarization percentage [%]
    BT_crit_data = BT_data[np.where(crit)]             # Brightness temperature [K]
    BT_slt_crit_data = BT_slt_data[np.where(crit)]     # Brightness temperature [K]
    BT_div_crit_data = BT_div_data[np.where(crit)]     # Brightness temperature ratio [NA]
    RA_crit_data = RA_data[np.where(crit)]             # RA [deg]
    Dec_crit_data = Dec_data[np.where(crit)]           # Dec [deg]
    
    # Get the uncertainties of PI, PA, and Per
    PI_err_crit_data = PI_err(Q_crit_data, U_crit_data, rms_Q, rms_Q)
    PA_err_crit_data = PA_err(Q_crit_data, U_crit_data, rms_Q, rms_Q)
    Per_err_crit_data = Per_err(I_crit_data, Q_crit_data, U_crit_data, \
                              rms_I, rms_Q, rms_Q, \
                              PI_err_crit_data)
    Per_err_crit_data = np.where(Per_err_crit_data>0.1, Per_err_crit_data, 0.1)
    
    
    return I_crit_data, Q_crit_data, U_crit_data, \
            PI_crit_data, PA_crit_data, Per_crit_data, \
            PI_err_crit_data, PA_err_crit_data, Per_err_crit_data, \
            BT_crit_data, BT_slt_crit_data, BT_div_crit_data, \
            RA_crit_data, Dec_crit_data
    
def Getdata(input_filename):
    # Input file is CASA format file with 4 dimension: (Stokes, freq, y, x)
    # Drop the dimension from 4 to 2: remain (y, x)    
    output_fits = pyfits.getdata(input_filename)[0][0]
    
    return output_fits
    
def PI_err(Q, U, Q_err, U_err):
    part1 = np.power(Q,2)*np.power(Q_err,2) + np.power(U,2)*np.power(U_err,2)
    part2 = np.power(Q,2) + np.power(U,2)
    return np.sqrt(np.divide(part1, part2))

def PA_err(Q, U, Q_err, U_err):
    part1 = np.power(Q,2)*np.power(U_err,2) + np.power(U,2)*np.power(Q_err,2)
    part2 = np.power(np.power(Q,2) + np.power(U,2) , 2)
    return 0.5*np.sqrt(np.divide(part1, part2))

def Per_err(I, Q, U, I_err, Q_err, U_err, PI_err):
    part1 = np.divide(np.power(PI_err,2), np.power(I,2))
    part2 = np.divide(np.power(I_err,2)*(np.power(Q,2) + np.power(U,2)), np.power(I,4))
    return np.sqrt(part1 + part2)*100

def GetTxtfile(outname, Select_region, SavetxtName, isMrt):
    
    # set the rms & clip critera from the different outname
    if outname == outname_b4:
        rms_I=rms_I_b4
        rms_Q=rms_Q_b4
        rms_PI=rms_PI_b4            
        I_clip=I_clip_b4
        PI_lowclip=PI_lowclip_b4
        PI_highclip=PI_highclip_b4
        sampling=sampling_b4
    elif outname == outname_b7:
        rms_I=rms_I_b7
        rms_Q=rms_Q_b7
        rms_PI=rms_PI_b7
        I_clip=I_clip_b7
        PI_lowclip=PI_lowclip_b7
        PI_highclip=PI_highclip_b7
        sampling=sampling_b7
            
    # Get the data
    I_crit_array, Q_crit_array, U_crit_array, \
    PI_crit_array, PA_crit_array, Per_crit_array, \
    PI_err_crit_array, PA_err_crit_array, Per_err_crit_array, \
    BT_crit_array, BT_slt_crit_array, BT_div_crit_array, \
    RA_crit_array, Dec_crit_array = \
    nyquist_sampling(outname, I_clip, PI_lowclip, PI_highclip, \
                     rms_I, rms_Q, rms_PI, sampling, Select_region)
    
    if isMrt == True:
        #Save the data to a txt file for machine readable table
        # - array_lens = np.shape(RA_crit_array)[0]
        # - type_arr = (j*np.ones((array_lens,), dtype=np.int))
        np.savetxt(SavetxtName, \
                   np.column_stack((RA_crit_array, Dec_crit_array,\
                                    I_crit_array, Q_crit_array, U_crit_array, \
                                    PI_crit_array, PA_crit_array, PA_err_crit_array, \
                                    Per_crit_array, Per_err_crit_array)),\
                   fmt='%.6f & %.6f & %.2g & %.2g & %.2g & %.2g & %.f & %.2g & %.2g & %.2g')
        
    else:  
        # Save the data to a txt file for data analysis
        np.savetxt(SavetxtName, \
                   np.column_stack((RA_crit_array, Dec_crit_array,\
                                    I_crit_array, Q_crit_array, U_crit_array,\
                                    PI_crit_array, PA_crit_array, Per_crit_array,\
                                    PI_err_crit_array, PA_err_crit_array, Per_err_crit_array,\
                                    BT_crit_array, BT_slt_crit_array, BT_div_crit_array,\
                                   )),\
                   fmt='%s & %s & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g & %.3g \\\\')

In [245]:
########################################################
# Main code
########################################################
if __name__ == "__main__":
    # Star measure time
    start_time = time.time()
    #---------------------------------------
    # Output the txt files from the observations
    
    # Decide the input outname
    outnames = [
                 outname_b4, outname_b7
                ]
    
    #----------------------------------------
    j = 0
    for outname in outnames:
        j = j+1
                
        #---------------------------------------------------------
        # All the selected pixel (4A1 & 4A2)
        
        # for data analysis
        Select_region = 'all'
        SavetxtName_all = '%snyquist_%s_all.txt'%(path_samping, outname) 
        GetTxtfile(outname, Select_region, SavetxtName_all, isMrt=False) 
        
        # machine readable table (mrt)
        SavetxtName_all_mrt = '%snyquist_%s_all_mrt.txt'%(path_samping, outname)
        GetTxtfile(outname, Select_region, SavetxtName_all_mrt, isMrt=True)  
        
        #---------------------------------------------------------
        # Split the 4A1 and 4A2
        
        # (1) 4A1 & 4A2: only the values within the masked B7 contour
        isMrt = False
        Select_region = 'mask'
        SavetxtName_mask = '%snyquist_%s_mask.txt'%(path_samping, outname)
        GetTxtfile(outname, Select_region, SavetxtName_mask, isMrt)  
       
        # (2) 4A1 only
        Select_region = 'mask_4A1'
        SavetxtName_mask_4A1 = '%snyquist_%s_mask_4A1.txt'%(path_samping, outname)
        GetTxtfile(outname, Select_region, SavetxtName_mask_4A1, isMrt)      

        # (3) 4A2 only
        Select_region = 'mask_4A2'
        SavetxtName_mask_4A2 = '%snyquist_%s_mask_4A2.txt'%(path_samping, outname)
        GetTxtfile(outname, Select_region, SavetxtName_mask_4A2, isMrt) 
       
        #---------------------------------------------------------
        # Inner100 AU: Only the values within certain radius
        
        nq_mask_list = np.loadtxt(SavetxtName_mask, dtype = str)
        nq_mask_4A1_list = np.loadtxt(SavetxtName_mask_4A1, dtype = str)
        nq_mask_4A2_list = np.loadtxt(SavetxtName_mask_4A2, dtype = str)
        num, b = np.shape(nq_mask_list)
        
        Nq_4A1_inner100au_array = []
        Nq_4A2_inner100au_array = []
        
        for i in xrange(num):
            
            ra_each = np.float(nq_mask_list[i][0])
            dec_each = np.float(nq_mask_list[i][2])
            
            #-----------------------------------------------------------
            # set the radius (e.g., 100au)
            radius_au = 100
            arcsec2deg = np.divide(1., 60*60)      # 1 arcsec [deg]
            AU2deg = np.divide(arcsec2deg, dist)   # 1 AU [deg]
            radius_deg = radius_au*AU2deg                # radius [AU in deg]
            
            #-----------------------------------------------------------
            # (4) innermost 100 AU region of IRAS4A1           
            dist_4A1 = np.sqrt(np.power(ra_4A1 - ra_each, 2) + np.power(dec_4A1 - dec_each, 2))

            if dist_4A1 <=  radius_deg:
                Nq_4A1_inner100au_array.append(nq_mask_list[i])

            #-----------------------------------------------------------
            # (5) innermost 100 AU region of IRAS4A2               
            dist_4A2 = np.sqrt(np.power(ra_4A2 - ra_each, 2) + np.power(dec_4A2 - dec_each, 2))

            if dist_4A2 <=  radius_deg:
                Nq_4A2_inner100au_array.append(nq_mask_list[i])


        SavetxtName_mask_4A1_inner100au = '%snyquist_%s_mask_4A1_inner100au.txt'%(path_samping, outname)
        SavetxtName_mask_4A2_inner100au = '%snyquist_%s_mask_4A2_inner100au.txt'%(path_samping, outname)
        
        np.savetxt(SavetxtName_mask_4A1_inner100au, Nq_4A1_inner100au_array, fmt="%s")
        np.savetxt(SavetxtName_mask_4A2_inner100au, Nq_4A2_inner100au_array, fmt="%s")

        
        #---------------------------------------------------------
        # Inner100 AU: Only the values within certain radius
        nq_mask_4A1_inner100au_list = np.loadtxt(SavetxtName_mask_4A1_inner100au, dtype = str)
        nq_mask_4A2_inner100au_list = np.loadtxt(SavetxtName_mask_4A2_inner100au, dtype = str)

        # (6) the surrounding region of IRAS4A1 but without inner 100 AU of 4A1
        A = [tuple(t) for t in nq_mask_4A2_list]
        B = [tuple(tt) for tt in nq_mask_4A2_inner100au_list]

        '''
        # Split the 4A1 and 4A2
        nyquist_name = '%snyquist_%s_slt.txt'%(path_samping, outname)
        nyquist_list = np.loadtxt(nyquist_name, dtype = str)
        num, b = np.shape(nyquist_list)
        
        Nq_4A1_inner100au_array = []
        Nq_4A2_inner100au_array = []
        Nq_4A1_near_array = []
        Nq_4A2_near_array = []
        Nq_4A1_near_wo_inner100au_array = []
        Nq_4A2_near_wo_inner100au_array = []
        
        for i in xrange(num):
            
            ra_each = np.float(nyquist_list[i][0])
            dec_each = np.float(nyquist_list[i][2])
            
            #-----------------------------------------------------------
            # set the data criteria
            arcsec2deg = np.divide(1., 60*60)      # 1 arcsec [deg]
            AU2deg = np.divide(arcsec2deg, dist)   # 1 AU [deg]
            radius_deg = 100*AU2deg                # radius [AU in deg]
            
            #-----------------------------------------------------------
            # (1) select the innermost 100 AU region of IRAS4A1           
            dist_4A1 = np.sqrt(np.power(ra_4A1 - ra_each, 2) + np.power(dec_4A1 - dec_each, 2))

            if dist_4A1 <=  radius_deg:
                Nq_4A1_inner100au_array.append(nyquist_list[i])

            #-----------------------------------------------------------
            # (2) select the innermost 100 AU region of IRAS4A2               
            dist_4A2 = np.sqrt(np.power(ra_4A2 - ra_each, 2) + np.power(dec_4A2 - dec_each, 2))

            if dist_4A2 <=  radius_deg:
                Nq_4A2_inner100au_array.append(nyquist_list[i])

            #-----------------------------------------------------------
            # (3) select the surrounding region of IRAS4A1
            # x=ra; y=dec
            ra_diff = np.abs(ra_4A1 - ra_4A2)
            dec_diff = np.abs(dec_4A1 - dec_4A2)       
            slope = np.divide(ra_diff, dec_diff)    
            #y = slope*(x-ra_4A1_4A2) + dec_4A1_4A2

            if dec_each < slope*(ra_each-ra_4A1_4A2) + dec_4A1_4A2:
                Nq_4A1_near_array.append(nyquist_list[i])

            #-----------------------------------------------------------
            # (4) select the surrounding region of IRAS4A2

            if dec_each >= slope*(ra_each-ra_4A1_4A2) + dec_4A1_4A2:
                Nq_4A2_near_array.append(nyquist_list[i])

            #-----------------------------------------------------------
            # (5) select the surrounding region of IRAS4A1 but without inner 100 AU of 4A1

            if dec_each < slope*(ra_each-ra_4A1_4A2) + dec_4A1_4A2 and dist_4A1 > radius_deg:
                Nq_4A1_near_wo_inner100au_array.append(nyquist_list[i])

            #-----------------------------------------------------------
            # (6) select the surrounding region of IRAS4A2 but without inner 100 AU of 4A1

            if dec_each >= slope*(ra_each-ra_4A1_4A2) + dec_4A1_4A2 and dist_4A2 > radius_deg:
                Nq_4A2_near_wo_inner100au_array.append(nyquist_list[i])
        
        np.savetxt('%snyquist_%s_4A1_inner100au.txt'%(path_samping, outname), Nq_4A1_inner100au_array, fmt="%s")
        np.savetxt('%snyquist_%s_4A2_inner100au.txt'%(path_samping, outname), Nq_4A2_inner100au_array, fmt="%s")
        np.savetxt('%snyquist_%s_4A1_nearby.txt'%(path_samping, outname), Nq_4A1_near_array, fmt="%s")
        np.savetxt('%snyquist_%s_4A2_nearby.txt'%(path_samping, outname), Nq_4A2_near_array, fmt="%s")
        np.savetxt('%snyquist_%s_4A1_nearby_wo_inner100au.txt'%(path_samping, outname), Nq_4A1_near_wo_inner100au_array, fmt="%s")
        np.savetxt('%snyquist_%s_4A2_nearby_wo_inner100au.txt'%(path_samping, outname), Nq_4A2_near_wo_inner100au_array, fmt="%s")
        '''
       
    # End measure time
    elapsed_time = time.time() - start_time
    print ('#----------------------------------------')
    print ("Done! Time spending = ", elapsed_time, "seconds.")

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in greater


#----------------------------------------
('Done! Time spending = ', 34.33142590522766, 'seconds.')


In [275]:
nq_mask_list = np.loadtxt(SavetxtName_mask, dtype = str)
nq_mask_4A1_list = np.loadtxt(SavetxtName_mask_4A1, dtype = str)
nq_mask_4A2_list = np.loadtxt(SavetxtName_mask_4A2, dtype = str)
nq_mask_4A1_inner100au_list = np.loadtxt(SavetxtName_mask_4A1_inner100au, dtype = str)
nq_mask_4A2_inner100au_list = np.loadtxt(SavetxtName_mask_4A2_inner100au, dtype = str)



num, b = np.shape(nq_mask_list)

print type(nq_mask_4A1_list)
print np.shape(nq_mask_4A1_list)
print nq_mask_4A1_list

A = [tuple(t) for t in nq_mask_4A2_list]
B = [tuple(tt) for tt in nq_mask_4A2_inner100au_list]
#print type(A)
#print type(set(A))

print np.shape(np.array(list(set(A)-set(B))))
print np.shape(np.array(list(set(A).difference(set(B)))))
print '---'
print np.shape(nq_mask_list)
print np.shape(nq_mask_4A1_list)
print np.shape(nq_mask_4A2_list)
print np.shape(nq_mask_4A1_inner100au_list)
print np.shape(nq_mask_4A2_inner100au_list)

<type 'numpy.ndarray'>
(752, 28)
[['52.293678316666146' '&' '31.22495833333256' ... '&' '0.892' '\\\\']
 ['52.293662074983075' '&' '31.22495833333564' ... '&' '0.874' '\\\\']
 ['52.293694558356385' '&' '31.22497222221637' ... '&' '0.876' '\\\\']
 ...
 ['52.293840734585274' '&' '31.225486110967488' ... '&' '0.994' '\\\\']
 ['52.29382449281148' '&' '31.225486110990957' ... '&' '0.984' '\\\\']
 ['52.29380825103772' '&' '31.225486111012394' ... '&' '0.951' '\\\\']]
(199, 28)
(199, 28)
---
(1069, 28)
(752, 28)
(317, 28)
(124, 28)
(118, 28)


In [222]:
outname = outname_b4
I_clip = I_clip_b4
rms_Q = rms_Q_b4
PI_lowclip = PI_lowclip_b4
PI_highclip = PI_highclip_b4


I_image = '%s%s_I.image.fits'%(path_image, outname)
Q_image = '%s%s_Q.image.fits'%(path_image, outname)
U_image = '%s%s_U.image.fits'%(path_image, outname)
PI_image = '%s%s_PI.image.fits'%(path_image, outname)
BT_image = '%s%s_BT.fits'%(path_BT, outname)

I_data = pyfits.getdata(I_image)[0][0]
Q_data = pyfits.getdata(Q_image)
U_data = pyfits.getdata(U_image)
I_hd = pyfits.getheader(I_image)
PI_data = pyfits.getdata(PI_image)
BT_data = pyfits.getdata(BT_image)

#print Q_data
#print U_data
#print rms_Q
PA_err_1 = PA_err(Q_data, U_data, rms_Q, rms_Q)
#print PA_err_1

print np.shape(BT_data)
# Get the coordinates (of world coor. system) of polarized intensity map
#wcs = WCS(I_hd)

#print wcs
# CASA simulation outputs have 4 dimension: (Stokes, freq, y, x)
#NS, Nf, Ny, Nx = I_data.shape
Ny, Nx = I_data.shape
nx, ny = np.meshgrid(np.arange(Nx), np.arange(Ny))

print Ny, Nx
print nx
print ny
'''
I_data = pyfits.getdata(I_image)[0][0]

crit_standd = (I_data > I_clip)

# sky = SkyCoord.from_pixel(nx, ny, wcs)
# print sky
# #print wcs.pixel_to_world_values

# i = 0
# j = 1
# RA_each = wcs.all_pix2world(nx[j, i], ny[j, i], 0, 0, 0)[0]
# Dec_each = wcs.all_pix2world(nx[j, i], ny[j, i], 0, 0, 0)[1]

#print wcs.all_pix2world(0, 1, 0, 0, 0)
#print wcs.all_pix2world(0, 2, 0, 0, 0)
#print wcs.all_pix2world(1, 2, 0, 0, 0)

#print RA_each
#print Dec_each


Coord = SkyCoord.from_pixel(nx, ny, wcs)
RA_data = Coord.ra.deg
Dec_data = Coord.dec.deg

I_crit_data = I_data[np.where(crit_standd)]
RA_crit_data = RA_data[np.where(crit_standd)]
'''

(1, 1, 3001, 3001)
3001 3001
[[   0    1    2 ... 2998 2999 3000]
 [   0    1    2 ... 2998 2999 3000]
 [   0    1    2 ... 2998 2999 3000]
 ...
 [   0    1    2 ... 2998 2999 3000]
 [   0    1    2 ... 2998 2999 3000]
 [   0    1    2 ... 2998 2999 3000]]
[[   0    0    0 ...    0    0    0]
 [   1    1    1 ...    1    1    1]
 [   2    2    2 ...    2    2    2]
 ...
 [2998 2998 2998 ... 2998 2998 2998]
 [2999 2999 2999 ... 2999 2999 2999]
 [3000 3000 3000 ... 3000 3000 3000]]


'\nI_data = pyfits.getdata(I_image)[0][0]\n\ncrit_standd = (I_data > I_clip)\n\n# sky = SkyCoord.from_pixel(nx, ny, wcs)\n# print sky\n# #print wcs.pixel_to_world_values\n\n# i = 0\n# j = 1\n# RA_each = wcs.all_pix2world(nx[j, i], ny[j, i], 0, 0, 0)[0]\n# Dec_each = wcs.all_pix2world(nx[j, i], ny[j, i], 0, 0, 0)[1]\n\n#print wcs.all_pix2world(0, 1, 0, 0, 0)\n#print wcs.all_pix2world(0, 2, 0, 0, 0)\n#print wcs.all_pix2world(1, 2, 0, 0, 0)\n\n#print RA_each\n#print Dec_each\n\n\nCoord = SkyCoord.from_pixel(nx, ny, wcs)\nRA_data = Coord.ra.deg\nDec_data = Coord.dec.deg\n\nI_crit_data = I_data[np.where(crit_standd)]\nRA_crit_data = RA_data[np.where(crit_standd)]\n'

In [205]:
print np.shape(RA_data)

RA_crit_data = RA_data[np.where(crit)]

(3001, 3001)


In [196]:
print type(sky)
print np.shape(sky)
print sky.ra.deg[0]
print np.shape(sky.ra.deg)
print sky.dec.deg[0]
print sky[0][0]
print sky[0][1]
print type(sky.ra.deg)
print type(sky.dec.deg)

<class 'astropy.coordinates.sky_coordinate.SkyCoord'>
(3001, 3001)
[52.29608204 52.29608042 52.2960788  ... 52.29121287 52.29121125
 52.29120962]
(3001, 3001)
[31.22333331 31.22333331 31.22333331 ... 31.22333331 31.22333331
 31.22333331]
<SkyCoord (ICRS): (ra, dec) in deg
    (52.29608204, 31.22333331)>
<SkyCoord (ICRS): (ra, dec) in deg
    (52.29608042, 31.22333331)>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [183]:
print type(sky)
print np.shape(sky)
#sky_arr = np.array(sky)
#print type(sky_arr)
#print np.shape(sky)


print wcs.all_pix2world(0, 0, 0, 0, 0)
print wcs.all_pix2world(0, 1, 0, 0, 0)
print wcs.all_pix2world(1, 0, 0, 0, 0)

print I_data[0][0][0]

print nx.ravel()
print np.shape(nx.ravel())
print ny.ravel()
print np.shape(ny.ravel())
print np.array([nx.ravel(), ny.ravel()])
nx_ny_arr = np.array([nx.ravel(), ny.ravel()]).T
print np.array([nx.ravel(), ny.ravel()]).T
print np.array([ny.ravel(), nx.ravel()]).T

wcs = WCS(I_hd)
#w, e = wcs.pixel_to_world_values(nx_ny_arr)

<class 'astropy.coordinates.sky_coordinate.SkyCoord'>
(3001, 3001)
[array(52.29608204), array(31.22333331), array(1.44985681e+11), array(1.)]
[array(52.29608204), array(31.2233347), array(1.44985681e+11), array(1.)]
[array(52.29608042), array(31.22333331), array(1.44985681e+11), array(1.)]
[ 3.1441232e-05  3.0045914e-05  2.8730663e-05 ... -1.5515131e-04
 -1.5828751e-04 -1.6135137e-04]
[   0    1    2 ... 2998 2999 3000]
(9006001,)
[   0    0    0 ... 3000 3000 3000]
(9006001,)
[[   0    1    2 ... 2998 2999 3000]
 [   0    0    0 ... 3000 3000 3000]]
[[   0    0]
 [   1    0]
 [   2    0]
 ...
 [2998 3000]
 [2999 3000]
 [3000 3000]]
[[   0    0]
 [   0    1]
 [   0    2]
 ...
 [3000 2998]
 [3000 2999]
 [3000 3000]]


AttributeError: 'WCS' object has no attribute 'pixel_to_world_values'

In [163]:
print I_data[0][0][0][0]
print I_data[0][0][0][1]
print I_data[0][0][1][0]
print sky[0][0]
print sky[0][1]
print sky[1][0]

3.1441232e-05
3.0045914e-05
3.033707e-05
<SkyCoord (ICRS): (ra, dec) in deg
    (52.29608204, 31.22333331)>
<SkyCoord (ICRS): (ra, dec) in deg
    (52.29608042, 31.22333331)>
<SkyCoord (ICRS): (ra, dec) in deg
    (52.29608204, 31.2233347)>


In [127]:
def PA_err_t(Q, U, Q_err, U_err):
    part1 = np.power(Q,2)*np.power(U_err,2) + np.power(U,2)*np.power(Q_err,2)
    part2 = np.power(np.power(Q,2) + np.power(U,2) , 2)
    return 0.5*np.sqrt(np.divide(part1, part2))

a1 = np.array([[np.nan, 1], [2,2]])
a2 = np.array([[2, 3], [4,5]])
a3 = np.array([[2, 3,9], [4,3,5], [4,7,5]])
print a3
a3 = a3[np.where(a3>3)]
print a3
print np.shape(a3)
print type(a3)
aa1 = 0.1
aa2 = 0.2

PA_t = PA_err_t(a1, a2, aa1, aa2)

Per_err_t = Per_err(a1, a1, a1, aa1, aa1, aa1, PA_t)



print PA_t
print Per_err_t

'''
    # Get the selected pixel value based on the criteria
    I_crit_data = np.where(crit, I_data, np.nan)
    Q_crit_data = np.where(crit, Q_data, np.nan)
    U_crit_data = np.where(crit, U_data, np.nan)
    PI_crit_data = np.where(crit, PI_data, np.nan)
    PA_crit_data = np.where(crit, PA_data, np.nan)
    Per_crit_data = np.where(crit, Per_data, np.nan)
    BT_crit_data = np.where(crit, BT_data, np.nan)
    BT_slt_crit_data = np.where(crit, BT_slt_data, np.nan)
    BT_div_crit_data = np.where(crit, BT_div_data, np.nan)
    
    PI_err_crit_data = PI_err(Q_crit_data, U_crit_data, rms_Q, rms_Q)
    PA_err_crit_data = PA_err(Q_crit_data, U_crit_data, rms_Q, rms_Q)
    Per_err_crit_data = Per_err(I_crit_data, Q_crit_data, U_crit_data, \
                              rms_I, rms_Q, rms_Q, \
                              PI_err_crit_data)
    Per_err_crit_data = np.where(Per_err_crit_data>0.1, Per_err_crit_data, 0.1)
'''

[[2 3 9]
 [4 3 5]
 [4 7 5]]
[9 4 5 4 7 5]
(6,)
<type 'numpy.ndarray'>
[[       nan 0.01802776]
 [0.01414214 0.01103987]]
[[        nan 14.25657743]
 [ 7.1063352   7.0925804 ]]


In [117]:
print type(I_data) # [ndarray]
print np.shape(I_data)
print I_data[0][0]

print np.where(I_data> I_clip, 1, 0)


condition = (I_data > I_clip) &\
            (PI_image > PI_lowclip) 

I_slt_image = np.where(condition, I_data, 0)
print I_slt_image[0][0][1500][1500]

x = np.array([[0,2],[3,5]])
print x
cond_1 = (x>1) & (x>0)
cond_2 = (x<4)
cond = cond_1 & cond_2
x = np.where(cond, x, np.nan)
print x

<type 'numpy.ndarray'>
(1, 1, 3001, 3001)
[[ 3.1441232e-05  3.0045914e-05  2.8730663e-05 ... -1.5515131e-04
  -1.5828751e-04 -1.6135137e-04]
 [ 3.0337071e-05  2.8967077e-05  2.7672510e-05 ... -1.5975349e-04
  -1.6282173e-04 -1.6580464e-04]
 [ 2.9217161e-05  2.7870112e-05  2.6593494e-05 ... -1.6432426e-04
  -1.6731539e-04 -1.7020856e-04]
 ...
 [-5.0146362e-05 -5.3674928e-05 -5.7023033e-05 ...  6.5391342e-04
   6.5538660e-04  6.5688533e-04]
 [-5.2946296e-05 -5.6385110e-05 -5.9631930e-05 ...  6.4934301e-04
   6.5081584e-04  6.5230217e-04]
 [-5.5625758e-05 -5.8972717e-05 -6.2116691e-05 ...  6.4478116e-04
   6.4625399e-04  6.4772909e-04]]
[[[[0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   ...
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]
   [0 0 0 ... 0 0 0]]]]
0.0018622006
[[0 2]
 [3 5]]
[[nan  2.]
 [ 3. nan]]


In [83]:
a = np.array([[1,2, 5], [-1,0, 5], [np.nan,0, 5]])
b = np.array([1,0, 5])
print a
print type(a)
print np.shape(a)
N, M = np.shape(a)
M_array = np.arange(M)
N_array = np.arange(N)
print N

print np.arange(N)
print M_array
print N_array

nx, ny = np.meshgrid(np.arange(N), np.arange(M))
print nx
print ny

sampling_interval = 3
print M_array % sampling_interval
print M_array % sampling_interval == 0

c = np.where((a>0) & (b>0), a, 0)
print c
condition = (a>0) & (b>0) & (M_array % sampling_interval == 0)
c = np.where(condition, a, 0)
#c = np.where((a>0) & (b>0) & (M_array % sampling_interval == 0), a, 0)
print c
print np.isnan(a)
#print not np.isnan(a)
print a
c = np.where((np.isnan(a) == False), a, 0)
print c

[[ 1.  2.  5.]
 [-1.  0.  5.]
 [nan  0.  5.]]
<type 'numpy.ndarray'>
(3, 3)
3
[0 1 2]
[0 1 2]
[0 1 2]
[[0 1 2]
 [0 1 2]
 [0 1 2]]
[[0 0 0]
 [1 1 1]
 [2 2 2]]
[0 1 2]
[ True False False]
[[1. 0. 5.]
 [0. 0. 5.]
 [0. 0. 5.]]
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[False False False]
 [False False False]
 [ True False False]]
[[ 1.  2.  5.]
 [-1.  0.  5.]
 [nan  0.  5.]]
[[ 1.  2.  5.]
 [-1.  0.  5.]
 [ 0.  0.  5.]]


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater


In [38]:
outname = outname_b4
I_image = '%s%s_I.fits'%(path_image, outname)
# Get the cube and header
I_data = pyfits.getdata(I_image)
I_hd = pyfits.getheader(I_image)

hd_CRVAL1 = float(I_hd['CRVAL1'])
hd_CDELT1 = float(I_hd['CDELT1'])
hd_CRVAL2 = float(I_hd['CRVAL2'])
hd_CDELT2 = float(I_hd['CDELT2'])
#hd_CRPIX1 = float(
print 'CRVAL1=', hd_CRVAL1, 'CRVAL2=', hd_CRVAL2
print 'CDELT1=', hd_CDELT1, 'CDELT2=', hd_CDELT2

wcs = WCS(I_hd)
# CASA simulation outputs have 4 dimension: (Stokes, freq, y, x)
NS, Nf, Ny, Nx = I_data.shape
nx, ny = np.meshgrid(np.arange(Nx), np.arange(Ny))


print NS, Nf, Ny, Nx
#print nx, ny
RA_slt_1d, Dec_slt_1d = [], []
print nx[4, 6]
print ny[4, 6]
print wcs.all_pix2world(800, 800, 0, 0, 0)
print 'RA'
print wcs.all_pix2world(800, 800, 0, 0, 0)[0] - wcs.all_pix2world(799, 799, 0, 0, 0)[0]
print wcs.all_pix2world(801, 801, 0, 0, 0)[0] - wcs.all_pix2world(800, 800, 0, 0, 0)[0]
print wcs.all_pix2world(802, 802, 0, 0, 0)[0] - wcs.all_pix2world(801, 801, 0, 0, 0)[0]
print 'Dec'
print wcs.all_pix2world(800, 800, 0, 0, 0)[1] - wcs.all_pix2world(799, 799, 0, 0, 0)[1]
print wcs.all_pix2world(801, 801, 0, 0, 0)[1] - wcs.all_pix2world(800, 800, 0, 0, 0)[1]
print wcs.all_pix2world(802, 802, 0, 0, 0)[1] - wcs.all_pix2world(801, 801, 0, 0, 0)[1]

print np.divide(wcs.all_pix2world(802, 802, 0, 0, 0)[0] - wcs.all_pix2world(801, 801, 0, 0, 0)[0] ,\
               wcs.all_pix2world(802, 802, 0, 0, 0)[1] - wcs.all_pix2world(801, 801, 0, 0, 0)[1])
#Dec_slt_1d.append(wcs.all_pix2world(nx[j, i], ny[j, i], 0, 0, 0)[1])
# for j in range(Ny):
#     for i in range(Nx):
#         # Get the RA and Dec in deg for each pixel
#         RA_slt_1d.append(wcs.all_pix2world(nx[j, i], ny[j, i], 0, 0, 0)[0])
#         Dec_slt_1d.append(wcs.all_pix2world(nx[j, i], ny[j, i], 0, 0, 0)[1])
print np.cos(np.deg2rad(hd_CRVAL2))            
print 2.*np.cos(np.deg2rad(hd_CRVAL2))            

CRVAL1= 52.2936459672 CRVAL2= 31.2254166013
CDELT1= -5.5555555543e-06 CDELT2= 5.5555555543e-06
1 1 1600 1600
6
4
[array(52.29364597), array(31.2254166), array(1.44985681e+11), array(1.)]
RA
-6.4967043513775025e-06
-6.49670511165823e-06
-6.496705871938957e-06
Dec
5.55555570258548e-06
5.555555400604817e-06
5.555555066649731e-06
-1.1694071598604072
0.8551343774759649
1.7102687549519298
